# Tesina Licenciatura
## Construcción de bases de trabajo

Este notebook constituye la primera parte del proceso de tratamiento de los datos.

En este, se filtra la información relativa a la producción petrolera mensual de México hasta construir la base de datos deseada para el posterior trabajo de estimación econométrica.

#### Importación de librerías necesarias y configuraciones iniciales

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
# Importación de librerías
import pandas as pd
import numpy as np
import datetime as dt
import glob
import re

### Carga de información

A partir de esta sección, se trabajará con la información de producción promedio mensual por pozo obtenida en el **SIH**. 

Lo primero a realizar es cargar correctamente la información.

In [3]:
### Producción petrolera --------------------------------------------------------------------

# Definir un path y listar los archivos referentes a producción y perforación en listas
path_prod = "/home/aleong/ITAM/tesina_licenciatura/Data/produccion/"

# A partir de la librería glob, se agrupan los archivos referentes a producción en una lista; y los de perforación en otra
extension = 'csv'
prod_filenames = [i for i in glob.glob(path_prod+'PROD_*.{}'.format(extension))]

# Los formatos de las variables contenidas en los csv de producción se van a definir con un diccionario
prod_type = (
    {
        'Cuenca':'object',
        'Asignación_o_Contrato':'object',
        'Nombre_del_pozo':'object',
        'Petróleo_(Mbd)':'float64',
        'Gas_asociado_(MMpcd)':'float64',
        'Gas__no_asociado_(MMpcd)':'float64',
        'Agua_(Mbd)':'float64',
        'Condensado_(Mbd)':'float64'
    }
)

# Carga de información de producción
df_prod = (
    pd.concat(
        [
            pd
            .read_csv(
                f,
                encoding='latin-1',
                skiprows=10,
                dtype=prod_type,
                parse_dates=['Fecha'],
                date_format="%d-%m-%Y"
            )
            for f in prod_filenames
        ],
        ignore_index=True
    )
)

# Cambio de nombre a columnas con base en un diccionario
prod_names = (
    {
        'Fecha':'fecha',
        'Cuenca':'cuenca',
        'Asignación_o_Contrato':'asignacion_contrato',
        'Nombre_del_pozo':'nombre_pozo',
        'Petróleo_(Mbd)':'petroleo_mbd',
        'Gas_asociado_(MMpcd)':'gas_asociado_mmpcd',
        'Gas_no_asociado_(MMpcd)':'gas_no_asociado_mmpcd',
        'Agua_(Mbd)':'agua_mbd',
        'Condensado_(Mbd)':'condensado_mbd'
    }
)

# Instrumentar cambio de nombre a columnas
df_prod.rename(columns=prod_names, inplace=True)

# Por simplicidad, se va a acotar el nombre de dos cuencas petroleras (CINTURÓN PLEGADO DE CHIAPAS Y CUENCAS DEL SURESTE)
# por "CHIAPAS" Y "SURESTE"
df_prod.loc[df_prod['cuenca'] == 'CINTURON PLEGADO DE CHIAPAS', 'cuenca'] = 'CHIAPAS'
df_prod.loc[df_prod['cuenca'] == 'CUENCAS DEL SURESTE', 'cuenca'] = 'SURESTE' 

# También se cambiará el nombre a un pozo, que requiere eliminar un espacio en blanco
df_prod.loc[df_prod['nombre_pozo'] == 'EL TREINTA-13DES', 'nombre_pozo'] = 'ELTREINTA-13DES'

In [4]:
### Producción petrolera --------------------------------------------------------------------

# Definir un path y listar los archivos referentes a producción y perforación en listas
path_perf = "/home/aleong/ITAM/tesina_licenciatura/Data/perforacion/" 

# Enlistar los archivos de perforación
perf_filenames = [i for i in glob.glob(path_perf+'PERFORACION_*.{}'.format(extension))]

# Los formatos de las variables contenidas en los csv de producción se van a definir con un diccionario
perf_type = (
    {
        'POZO':'object',
        'CAMPO':'object',
        'ENTIDAD':'object',
        'UBICACIóN':'object',
        'CLASIFICACIóN':'object',
        'ESTADO ACTUAL':'object',
        'TIPO DE HIDROCARBURO':'object',
        'FECHA INICIO DE PERFORACIóN':'object',
        'FECHA FIN PERFORACIóN':'object',
        'PROFUNDIDAD TOTAL':'float64',
        'PROFUNDIDAD VERTICAL':'float64',
        'TRAYECTORIA':'object',
        'DISPONIBLE':'object'
    }
)

# Carga de información de producción
df_perf = (
    pd.concat(
        [
            pd
            .read_csv(
                f,
                encoding='utf-8-sig',
                dtype=perf_type,
                parse_dates={'fecha_inicio':[7], 'fecha_fin':[8]},
                date_format="%d-%m-%Y"
            )
            for f in perf_filenames
        ],
        ignore_index=True
    )
)

# Cambio de nombre a columnas con base en un diccionario
perf_names = (
    {
        'POZO':'nombre_pozo',
        'CAMPO':'campo',
        'ENTIDAD':'entidad',
        'UBICACIóN':'ubicacion',
        'CLASIFICACIóN':'clasificacion',
        'ESTADO ACTUAL':'edo_actual',
        'TIPO DE HIDROCARBURO':'tipo_hidrocarburo',
        'PROFUNDIDAD TOTAL':'prof_total',
        'PROFUNDIDAD VERTICAL':'prof_vertical',
        'TRAYECTORIA':'trayectoria',
        'DISPONIBLE':'disponible'
    }
)


# Cambios de nombre de columnas
df_perf.rename(columns=perf_names, inplace=True)


**¿Cuántos pozos hay registrados en la base de datos de producción mensual promedio de petróleo?**

In [5]:
# Número inicial de pozos
n0_pozos = df_prod['nombre_pozo'].nunique()

# Imprimir conteo inicial
(
    print(
        f"""
    BASE DE PRODUCCIÓN PETROLERA
    ----------------------------

        Número de pozos:        {n0_pozos:>7,}
        Cuencas petroleras:     {df_prod['cuenca'].nunique():>7,}

        ------------------------

        Inicio:                 {(df_prod['fecha'].dt.year).min():>7}
        Final:                  {(df_prod['fecha'].dt.year).max():>7}

        """
    )
)


    BASE DE PRODUCCIÓN PETROLERA
    ----------------------------

        Número de pozos:         20,756
        Cuencas petroleras:           6

        ------------------------

        Inicio:                    1930
        Final:                     2020

        


### Definición del universo de pozos

Ahora, se van a aplicar los siguientes filtros sobre este universo de pozos:  

1. **Descarte de pozos no petroleros.**
2. **Descarte de pozos previos a 1970.**
3. **Descarte de pozos posteriores a 2018.**

Pero, para su aplicación efectiva sobre la base de producción petrolera, antes se creará un **inventario de pozos** que reuna, para cada pozos, la siguiente información: **inicio de producción, la cuenca, el tipo de hidrocarburo y la ubicación del pozo.**

#### Inventario de pozos

In [6]:
# El inventario de pozos se crea a partir de agrupar la base de producción por pozo y ejecutar
# algunas operaciones a nivel agregado. Primero, se agregará el inicio de producción y la cuenca
inv_pozos = (
    df_prod
    .groupby(['nombre_pozo'], as_index=False)
    .agg(
        inicio_prod=('fecha', 'min'),
        cuenca=('cuenca', 'first')
    )
)

# Con la ayuda de la base de perforación proveniente del mapa, incluir el tipo de hidrocarburo, 
# la ubicación y la entidad
inv_pozos = (
    inv_pozos
    .merge(
        df_perf[['nombre_pozo', 'tipo_hidrocarburo', 'ubicacion']],
        on=['nombre_pozo'],
        how='left'
    )
)

En este cruce, se generan algunos registros duplicados. Para remediar esto, solo se conservará un registro por pozo.

In [7]:
# Número de pozos duplicados
pozos_dup = inv_pozos[inv_pozos['nombre_pozo'].duplicated()]['nombre_pozo'].nunique()

# Elminar los registros duplicados
inv_pozos.drop_duplicates(subset=['nombre_pozo'], keep='last', inplace=True)

# ¿Restan duplicados?
if len(inv_pozos) == inv_pozos['nombre_pozo'].nunique():
    
    print("Ya no hay pozos duplicados.")
    
else:
    
    print("Revisar de nuevo.")

Ya no hay pozos duplicados.


#### Filtro 1: descarte de pozos no petroleros

Primero, identificar el tipo de hidrocarburo producido en cada pozo a partir de la columna "tipo_hidrocarburo" (proveniente de la base de perforación)

In [8]:
# Se crea una dummy para mapear la columna de tipo de hidrocarburo
inv_pozos['petrolero'] = (
    inv_pozos['tipo_hidrocarburo']
    .map(
        {
            'ACEITE':1,
            'ACEITE-GAS':1,
            'ACEITE Y GAS':1,
            'ACEITE SUPERLIGERO':1,
            'ACEITE LIGERO':1,
            'ACEITE NEGRO':1,
            'GAS':0,
            'GAS-CONDENSADO':0,
            'CONDENSADO':0,
            'GAS SECO':0,
            'GAS HUMEDO':0,
            'GAS Y CONDENSADO':0,
            'NO IDENTIFICADO':0
        }
    )
)

# ¿Cuántos faltan por identificar?:
print(
    f"""    
    Pozos por identificar:    {inv_pozos['petrolero'].isna().sum():>7,}
    """
)

    
    Pozos por identificar:        359
    


Para identificar los restantes, se va a aplicar como criterio que, si en la base de producción el pozo produjo, al menos en uno de los meses, una cantidad de petróleo estrictamente mayor a cero.

In [9]:
# Primero, tomar el nombre de los pozos restantes por identificar y almacenarlo en una lista
pending = inv_pozos[inv_pozos['petrolero'].isna()]['nombre_pozo'].tolist()

# Tomar estos pozos de la base de producción
pozos_pendientes = (
    df_prod[df_prod['nombre_pozo'].isin(pending)]
    .groupby(['nombre_pozo'], as_index=False)
    .agg(
        agg_prod=('petroleo_mbd','sum')
    )
)

# Separar aquellos que sean petroleros
pozos_pendientes = pozos_pendientes[pozos_pendientes['agg_prod']>0]['nombre_pozo'].tolist()

# Ahora, actualizar esta información en el inventario de pozos
inv_pozos.loc[inv_pozos['nombre_pozo'].isin(pozos_pendientes), 'petrolero'] = 1

# ¿Cuántos faltan por identificar?:
print(
    f"""
    Pozos por identificar:    {inv_pozos['petrolero'].isna().sum():>7,}
    """
)


    Pozos por identificar:         48
    


Finalmente, los 49 pozos restantes no serán considerados petroleros.

In [10]:
# Imputar los pozos restantes como cero en la columna 'petrolero' del inventario de pozos
inv_pozos.loc[inv_pozos['petrolero'].isna(), 'petrolero'] = 0

# Cambiar el formato de esta misma columna a integer
inv_pozos['petrolero'] = inv_pozos['petrolero'].astype('int64')

# ¿Cuántos pozos faltan por identificar?
print(
    f"""
    INVENTARIO DE POZOS
    -------------------
    
    Pozos identificados:      {inv_pozos['petrolero'].notna().sum():>7,}
    
        Petroleros:           {inv_pozos['petrolero'].value_counts()[1]:>7,}
        Otros:                {inv_pozos['petrolero'].value_counts()[0]:>7,}
    
    ----------------------------------------
    
    Pozos por identificar:    {inv_pozos['petrolero'].isna().sum():>7,}
    
    """
)

# Contar el número de pozos restante
n1_pozos = inv_pozos[inv_pozos['petrolero']==1]['nombre_pozo'].nunique()
no_petroleros = inv_pozos[inv_pozos['petrolero']==0]['nombre_pozo'].nunique()


    INVENTARIO DE POZOS
    -------------------
    
    Pozos identificados:       20,756
    
        Petroleros:            13,958
        Otros:                  6,798
    
    ----------------------------------------
    
    Pozos por identificar:          0
    
    


#### Filtro 2: descarte de pozos previos a 1970

El siguiente paso en el proceso de selección de pozos es identificar aquellos que empezaron a **producir a partir de enero de 1970.**

In [11]:
# Creación de una marca en el inventario de pozos
inv_pozos['post_1970'] = np.where(inv_pozos['inicio_prod']>="1970-01-01", 1, 0)

# ¿Cuántos pozos son pre-1970?
pre_1970 = inv_pozos[(inv_pozos['post_1970']==0)&(inv_pozos['petrolero']==1)]['nombre_pozo'].nunique()
post_1970 = inv_pozos[(inv_pozos['post_1970']==1)&(inv_pozos['petrolero']==1)]['nombre_pozo'].nunique()

# Conteo de pozos
n2_pozos = inv_pozos[(inv_pozos['post_1970']==1)&(inv_pozos['petrolero']==1)]['nombre_pozo'].nunique()

#### Filtro 3: descarte de pozos posteriores a 2018

El último filtro a aplicar consiste en eliminar aquellos **pozos que empezaron a producir después de diciembre de 2018.**

In [12]:
# Creación de una marca en el inventario de pozos
inv_pozos['post_2018'] = np.where(inv_pozos['inicio_prod']>="2019-01-01", 1, 0)

# ¿Cuántos pozos son post-2018?
pre_2018 = inv_pozos[(inv_pozos['post_2018']==0) & (inv_pozos['post_1970']==1)]['nombre_pozo'].nunique()
post_2018 = inv_pozos[(inv_pozos['post_2018']==1) & (inv_pozos['post_1970']==1) & (inv_pozos['petrolero']==1)]['nombre_pozo'].nunique()

# Conteo de pozos
n3_pozos = (
    inv_pozos[(inv_pozos['post_1970']==1)&(inv_pozos['petrolero']==1) & (inv_pozos['post_2018']==0)]['nombre_pozo'].nunique()
)

**¿Cuántos pozos restan?**

A continuación, se muestra un resumen del proceso

In [13]:
print(
    f"""
    BASE DE PRODUCCIÓN PETROLERA
    ----------------------------

    Número inicial de pozos:      {n0_pozos:>7,}  (- {no_petroleros:>5,} pozos no petroleros)
    Pozos petroleros:             {n1_pozos:>7,}  (- {pre_1970:>5,} pozos pre-1970)
    Pozos post 1970:              {n2_pozos:>7,}  (- {post_2018:>5,} pozos post-2018)
    ______________________________________
    
    Pozos restantes:              {n3_pozos:>7,}
    """
)


    BASE DE PRODUCCIÓN PETROLERA
    ----------------------------

    Número inicial de pozos:       20,756  (- 6,798 pozos no petroleros)
    Pozos petroleros:              13,958  (- 2,007 pozos pre-1970)
    Pozos post 1970:               11,951  (-   295 pozos post-2018)
    ______________________________________
    
    Pozos restantes:               11,656
    


Finalmente, se escriben el universo de pozos y el inventario creado en dos archivos csv separados.

In [14]:
# Crear tres máscaras para filtrar los datos de producción
pozos_petroleros = (inv_pozos['petrolero']==1)
post_1970 = (inv_pozos['post_1970']==1)
pre_2018 = (inv_pozos['post_2018']==0)

# Lista final de pozos a seleccionar
lista_pozos = inv_pozos[pozos_petroleros & post_1970 & pre_2018]['nombre_pozo'].tolist()

# Filtrar la producción de estos pozos
universo_pozos = df_prod[df_prod['nombre_pozo'].isin(lista_pozos)].copy()

# Finalmente, acotar los registros hasta diciembre de 2018
universo_pozos = universo_pozos[universo_pozos['fecha']<"2019-01-01"]

# Crear los csv
inv_pozos.to_csv("/home/aleong/ITAM/tesina_licenciatura/Data/bases_finales/inventario_pozos.csv", sep=',', index=False)
universo_pozos.to_csv("/home/aleong/ITAM/tesina_licenciatura/Data/bases_finales/universo_pozos.csv", sep=',', index=False)